<a href="https://colab.research.google.com/github/aritam11/ChatApp/blob/master/CNN%2BRNN%2CCNN%2BLSTM%2CCNN%2BGRU_with_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from wordcloud import WordCloud

In [153]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, Conv1D, LSTM , MaxPool1D , GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [154]:
fake = pd.read_csv("Fake.csv")

In [155]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [156]:
fake["subject"].value_counts()

News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64

In [157]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [158]:
real = pd.read_csv("True.csv")
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [159]:
real["subject"].value_counts()

politicsNews    11272
worldnews       10145
Name: subject, dtype: int64

In [160]:
unknown_publishers = []
for index,row in enumerate(real.text.values):
    try:
        record = row.split(" -", maxsplit=1)
        #if no text part is present, following will give error
        record[1]
        #if len of piblication part is greater than 260
        #following will give error, ensuring no text having "-" in between is counted
        assert(len(record[0]) < 260)
    except:
        unknown_publishers.append(index)

In [161]:
real.iloc[unknown_publishers].text

3488     The White House on Wednesday disclosed a group...
4358     Neil Gorsuch, President Donald Trump’s appoint...
4465     WASHINGTON The clock began running out this we...
5784     Federal appeals court judge Neil Gorsuch, the ...
6660     Republican members of Congress are complaining...
6823     Over the course of the U.S. presidential campa...
7922     After going through a week reminiscent of Napo...
8194     The following timeline charts the origin and s...
8195     Global health officials are racing to better u...
8247     U.S. President Barack Obama visited a street m...
8465     ALGONAC, MICH.—Parker Fox drifted out of the D...
8481     Global health officials are racing to better u...
8482     The following timeline charts the origin and s...
8505     Global health officials are racing to better u...
8506     The following timeline charts the origin and s...
8771     In a speech weighted with America’s complicate...
8970                                                    

In [162]:
publisher = []
tmp_text = []
for index,row in enumerate(real.text.values):
    if index in unknown_publishers:
        #Add unknown of publisher not mentioned
        tmp_text.append(row)
        
        publisher.append("Unknown")
        continue
    record = row.split(" -", maxsplit=1)
    publisher.append(record[0])
    tmp_text.append(record[1])

In [163]:
real["publisher"] = publisher
real["text"] = tmp_text

del publisher, tmp_text, record, unknown_publishers

In [164]:
real.head()

,title,text,subject,date,publisher
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction...,politicsNews,"December 31, 2017",WASHINGTON (Reuters)
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fi...,politicsNews,"December 29, 2017",WASHINGTON (Reuters)
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links be...,politicsNews,"December 31, 2017",WASHINGTON (Reuters)
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos to...,politicsNews,"December 30, 2017",WASHINGTON (Reuters)
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Pos...,politicsNews,"December 29, 2017",SEATTLE/WASHINGTON (Reuters)


In [165]:
[index for index,text in enumerate(real.text.values) if str(text).strip() == '']

[8970]

In [166]:
real = real.drop(8970, axis=0)

In [167]:
# checking for the same in fake news
empty_fake_index = [index for index,text in enumerate(fake.text.values) if str(text).strip() == '']
print(f"No of empty rows: {len(empty_fake_index)}")
fake.iloc[empty_fake_index].tail()

No of empty rows: 630


,title,text,subject,date
21816,BALTIMORE BURNS: MARYLAND GOVERNOR BRINGS IN N...,,left-news,"Apr 27, 2015"
21826,FULL VIDEO: THE BLOCKBUSTER INVESTIGATION INTO...,,left-news,"Apr 25, 2015"
21827,(VIDEO) HILLARY CLINTON: RELIGIOUS BELIEFS MUS...,,left-news,"Apr 25, 2015"
21857,(VIDEO)ICE PROTECTING OBAMA: WON’T RELEASE NAM...,,left-news,"Apr 14, 2015"
21873,(VIDEO) HYSTERICAL SNL TAKE ON HILLARY’S ANNOU...,,left-news,"Apr 12, 2015"


In [168]:
real["class"] = 1
fake["class"] = 0

In [169]:
#Combining Title and Text
real["text"] = real["title"] + " " + real["text"]
fake["text"] = fake["title"] + " " + fake["text"]

In [170]:
real = real.drop(["subject", "date","title",  "publisher"], axis=1)
fake = fake.drop(["subject", "date", "title"], axis=1)

In [171]:
data = real.append(fake, ignore_index=True)
del real, fake

In [172]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Removing StopWords, Punctuations and single-character words

In [173]:
y = data["class"].values
#Converting X to format acceptable by gensim, removing annd punctuation stopwords in the process
X = []
stop_words = set(nltk.corpus.stopwords.words("english"))
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in data["text"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
        tmp.extend(filtered_words)
    X.append(tmp)

del data

In [174]:
import gensim

In [175]:
#Dimension of vectors we are generating
EMBEDDING_DIM = 100

#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences=X,size=EMBEDDING_DIM, window=5, min_count=1)

In [177]:
len(w2v_model.wv.key_to_index)

AttributeError: ignored

In [178]:
w2v_model.wv.most_similar("Army")

[('Marine', 0.74330735206604),
 ('Corps', 0.6753474473953247),
 ('Navy', 0.6678800582885742),
 ('Guard', 0.6573798060417175),
 ('commander', 0.6398434638977051),
 ('Attahiru', 0.6151527166366577),
 ('commanders', 0.6140105724334717),
 ('Grassrope', 0.606911838054657),
 ('army', 0.6029186248779297),
 ('admirals', 0.6017786264419556)]

In [179]:
w2v_model.wv.most_similar("Iran")

[('Tehran', 0.8924319744110107),
 ('Iranian', 0.757009744644165),
 ('Hezbollah', 0.6626573801040649),
 ('Riyadh', 0.6615345478057861),
 ('nuclear', 0.652499258518219),
 ('Iranians', 0.6247107982635498),
 ('destabilizing', 0.6135573387145996),
 ('Turkey', 0.6092820167541504),
 ('Antagonize', 0.6075030565261841),
 ('Saudis', 0.6072289943695068)]

In [180]:
w2v_model.wv.most_similar("FBI")

[('Comey', 0.7159454226493835),
 ('CIA', 0.6284183263778687),
 ('bureau', 0.6118370294570923),
 ('investigators', 0.6090315580368042),
 ('Mueller', 0.6019518971443176),
 ('DOJ', 0.5934025645256042),
 ('Investigation', 0.5574303865432739),
 ('investigation', 0.5545222759246826),
 ('investigations', 0.5516840219497681),
 ('inquiry', 0.5467161536216736)]

In [181]:
w2v_model.wv.most_similar("Facebook")

[('Twitter', 0.7097092866897583),
 ('Instagram', 0.6733429431915283),
 ('Google', 0.6556414365768433),
 ('posting', 0.6294727921485901),
 ('Reddit', 0.6244467496871948),
 ('website', 0.6231716871261597),
 ('GoFundMe', 0.61151123046875),
 ('blog', 0.601448655128479),
 ('Zuckerberg', 0.5858651399612427),
 ('FB', 0.5727802515029907)]

In [182]:
w2v_model.wv.most_similar(positive=["Trump","Obama", "Clinton"])

[('Cruz', 0.5597764849662781),
 ('His', 0.5505765080451965),
 ('Bush', 0.5250269770622253),
 ('He', 0.5148721933364868),
 ('Hillary', 0.511627733707428),
 ('She', 0.4857006072998047),
 ('Putin', 0.46154409646987915),
 ('incoming', 0.4604354500770569),
 ('publicly', 0.4577462375164032),
 ('Comey', 0.4575670063495636)]

In [183]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

In [184]:
word_index = tokenizer.word_index
for word, num in word_index.items():
    print(f"{word} : {num}")
    if num == 10:
        break  

trump : 1
said : 2
the : 3
president : 4
would : 5
people : 6
one : 7
state : 8
new : 9
it : 10


In [185]:
nos = np.array([len(x) for x in X])
len(nos[nos  < 700])

43828

In [186]:
maxlen = 700 

#Making all news of size maxlen defined above
X = pad_sequences(X, maxlen=maxlen)


In [187]:
#+1 for UNKNOWN
vocab_size = len(tokenizer.word_index) + 1

In [188]:
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        try:
            weight_matrix[i] = model.wv[word]
        except:
            weight_matrix[i] = np.zeros(EMBEDDING_DIM)
    return weight_matrix

In [189]:
embedding_vectors = get_weight_matrix(w2v_model, word_index)

# CNN + RNN

In [190]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
#CNN
model.add(Conv1D(activation='relu', filters=4, kernel_size=4))
model.add(MaxPool1D())
#RNN
model.add(SimpleRNN(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [191]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 700, 100)          12239100  
                                                                 
 conv1d_11 (Conv1D)          (None, 697, 4)            1604      
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 348, 4)           0         
 g1D)                                                            
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 128)               17024     
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 12,257,857
Trainable params: 18,757
Non-trainable params: 12,239,100
____________________________________

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [193]:
rnn_hist = model.fit(X_train, y_train, validation_split=0.3, epochs=2)

Epoch 1/2
737/737 [==============================] - 117s 157ms/step - loss: 0.2637 - acc: 0.8983 - val_loss: 0.2107 - val_acc: 0.9275
Epoch 2/2
737/737 [==============================] - 117s 159ms/step - loss: 0.2264 - acc: 0.9128 - val_loss: 0.3065 - val_acc: 0.8664


In [194]:
y_pred = (model.predict(X_test) >= 0.5).astype("int")

In [195]:
accuracy_score(y_test, y_pred)

0.8599554565701559

In [196]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.84      0.86      5842
           1       0.83      0.89      0.86      5383

    accuracy                           0.86     11225
   macro avg       0.86      0.86      0.86     11225
weighted avg       0.86      0.86      0.86     11225



# CNN + LSTM

In [197]:
#Defining Neural Network
model1 = Sequential()
#Non-trainable embeddidng layer
model1.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
#CNN
model1.add(Conv1D(activation='relu', filters=4, kernel_size=4))
model1.add(MaxPool1D())
#LSTM
model1.add(LSTM(128))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [198]:
model1.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 700, 100)          12239100  
                                                                 
 conv1d_12 (Conv1D)          (None, 697, 4)            1604      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 348, 4)           0         
 g1D)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 128)               68096     
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 12,308,929
Trainable params: 69,829
Non-trainable params: 12,239,100
____________________________________

In [199]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [200]:
lstm_hist = model.fit(X_train, y_train, validation_split=0.3, epochs=2)

Epoch 1/2
737/737 [==============================] - 115s 155ms/step - loss: 0.2930 - acc: 0.8750 - val_loss: 0.2703 - val_acc: 0.8857
Epoch 2/2
737/737 [==============================] - 115s 157ms/step - loss: 0.2283 - acc: 0.9086 - val_loss: 0.1850 - val_acc: 0.9337


In [201]:
y_pred = (model.predict(X_test) >= 0.5).astype("int")

In [202]:
accuracy_score(y_test, y_pred)

0.9278396436525612

In [203]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      5855
           1       0.91      0.94      0.93      5370

    accuracy                           0.93     11225
   macro avg       0.93      0.93      0.93     11225
weighted avg       0.93      0.93      0.93     11225



# CNN + GRU

In [204]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
model.add(Conv1D(activation='relu', filters=4, kernel_size=4))
model.add(MaxPool1D())
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])




In [205]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [206]:
gru_hist = model.fit(X_train, y_train, validation_split=0.3, epochs=2)

Epoch 1/2
737/737 [==============================] - 228s 302ms/step - loss: 0.1825 - acc: 0.9280 - val_loss: 0.0904 - val_acc: 0.9682
Epoch 2/2
737/737 [==============================] - 220s 299ms/step - loss: 0.0740 - acc: 0.9727 - val_loss: 0.0607 - val_acc: 0.9784


In [207]:
y_pred = (model.predict(X_test) >= 0.5).astype("int")

In [208]:
accuracy_score(y_test, y_pred)

0.97728285077951

In [209]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      5877
           1       0.98      0.97      0.98      5348

    accuracy                           0.98     11225
   macro avg       0.98      0.98      0.98     11225
weighted avg       0.98      0.98      0.98     11225



In [210]:
accuracy_rnn = ['CNN+RNN', 
               np.mean(rnn_hist.history['acc']), 
               np.mean(rnn_hist.history['val_acc'])]

accuracy_lstm = ['CNN+LSTM', 
                np.mean(lstm_hist.history['acc']), 
                np.mean(lstm_hist.history['val_acc'])]

accuracy_gru = ['CNN+GRU', 
                np.mean(gru_hist.history['acc']), 
                np.mean(gru_hist.history['val_acc'])]

In [221]:
comparison = pd.DataFrame([accuracy_rnn])
# append all other scores
comparison = comparison.append([accuracy_lstm,accuracy_gru])

In [222]:
comparison.columns = ['Algorithm', 'Training Accuracy', 'Validation Accuracy']
comparison.set_index(['Algorithm'], inplace=True)
comparison

,Training Accuracy,Validation Accuracy
Algorithm,,
CNN+RNN,0.905537,0.896951
CNN+LSTM,0.891790,0.909671
CNN+GRU,0.950318,0.973322
